<h1>Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [16]:
!pip install google.cloud

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

2.2.0


Read data created in the previous chapter.

In [4]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [5]:
def make_train_input_fn(df, num_epochs):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [6]:
def make_eval_input_fn(df):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [7]:
def make_prediction_input_fn(df):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [8]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [9]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [12]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-16T15:52:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.60989s
INFO:tensorflow:Finished evaluation at 2020-07-16-15:52:14
INFO:tensorflow:Saving dict for global step 6071: average_loss = 109.16252, global_

This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [13]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.099371], dtype=float32)}
{'predictions': array([11.098193], dtype=float32)}
{'predictions': array([11.097498], dtype=float32)}
{'predictions': array([11.094409], dtype=float32)}
{'predictions': array([11.0979595], dtyp

{'predictions': array([11.096427], dtype=float32)}
{'predictions': array([11.097024], dtype=float32)}
{'predictions': array([11.0740385], dtype=float32)}
{'predictions': array([11.098002], dtype=float32)}
{'predictions': array([11.077848], dtype=float32)}
{'predictions': array([11.076986], dtype=float32)}
{'predictions': array([11.099612], dtype=float32)}
{'predictions': array([11.098654], dtype=float32)}
{'predictions': array([11.0782585], dtype=float32)}
{'predictions': array([11.071693], dtype=float32)}
{'predictions': array([11.077478], dtype=float32)}
{'predictions': array([11.096078], dtype=float32)}
{'predictions': array([11.089221], dtype=float32)}
{'predictions': array([11.055771], dtype=float32)}
{'predictions': array([11.096193], dtype=float32)}
{'predictions': array([11.098532], dtype=float32)}
{'predictions': array([11.09871], dtype=float32)}
{'predictions': array([11.076603], dtype=float32)}
{'predictions': array([11.055502], dtype=float32)}
{'predictions': array([11.0987

{'predictions': array([11.097037], dtype=float32)}
{'predictions': array([11.03427], dtype=float32)}
{'predictions': array([11.09812], dtype=float32)}
{'predictions': array([11.098508], dtype=float32)}
{'predictions': array([11.066806], dtype=float32)}
{'predictions': array([11.094447], dtype=float32)}
{'predictions': array([11.098089], dtype=float32)}
{'predictions': array([11.077946], dtype=float32)}
{'predictions': array([11.093523], dtype=float32)}
{'predictions': array([10.99039], dtype=float32)}
{'predictions': array([11.098787], dtype=float32)}
{'predictions': array([11.0986], dtype=float32)}
{'predictions': array([11.09654], dtype=float32)}
{'predictions': array([11.097139], dtype=float32)}
{'predictions': array([11.076615], dtype=float32)}
{'predictions': array([11.098232], dtype=float32)}
{'predictions': array([11.096984], dtype=float32)}
{'predictions': array([11.098057], dtype=float32)}
{'predictions': array([11.055817], dtype=float32)}
{'predictions': array([11.097039], dt

{'predictions': array([11.095495], dtype=float32)}
{'predictions': array([11.036201], dtype=float32)}
{'predictions': array([11.098568], dtype=float32)}
{'predictions': array([11.097874], dtype=float32)}
{'predictions': array([11.097007], dtype=float32)}
{'predictions': array([11.077356], dtype=float32)}
{'predictions': array([11.09473], dtype=float32)}
{'predictions': array([11.013909], dtype=float32)}
{'predictions': array([11.096915], dtype=float32)}
{'predictions': array([11.097265], dtype=float32)}
{'predictions': array([11.0965805], dtype=float32)}
{'predictions': array([11.0979595], dtype=float32)}
{'predictions': array([11.095967], dtype=float32)}
{'predictions': array([11.09633], dtype=float32)}
{'predictions': array([11.056819], dtype=float32)}
{'predictions': array([11.095734], dtype=float32)}
{'predictions': array([11.0959835], dtype=float32)}
{'predictions': array([11.096911], dtype=float32)}
{'predictions': array([11.096592], dtype=float32)}
{'predictions': array([11.0366

{'predictions': array([11.096511], dtype=float32)}
{'predictions': array([10.994033], dtype=float32)}
{'predictions': array([11.01162], dtype=float32)}
{'predictions': array([11.099092], dtype=float32)}
{'predictions': array([11.097844], dtype=float32)}
{'predictions': array([11.077427], dtype=float32)}
{'predictions': array([11.0152235], dtype=float32)}
{'predictions': array([11.09736], dtype=float32)}
{'predictions': array([11.077004], dtype=float32)}
{'predictions': array([11.097594], dtype=float32)}
{'predictions': array([11.075889], dtype=float32)}
{'predictions': array([11.097744], dtype=float32)}
{'predictions': array([11.091266], dtype=float32)}
{'predictions': array([11.072797], dtype=float32)}
{'predictions': array([11.097225], dtype=float32)}
{'predictions': array([11.055429], dtype=float32)}
{'predictions': array([11.09908], dtype=float32)}
{'predictions': array([11.098083], dtype=float32)}
{'predictions': array([11.097751], dtype=float32)}
{'predictions': array([11.015164]

{'predictions': array([11.097203], dtype=float32)}
{'predictions': array([11.035809], dtype=float32)}
{'predictions': array([11.097088], dtype=float32)}
{'predictions': array([11.0783825], dtype=float32)}
{'predictions': array([11.098124], dtype=float32)}
{'predictions': array([11.098468], dtype=float32)}
{'predictions': array([11.075895], dtype=float32)}
{'predictions': array([11.098723], dtype=float32)}
{'predictions': array([11.014291], dtype=float32)}
{'predictions': array([11.096384], dtype=float32)}
{'predictions': array([11.096642], dtype=float32)}
{'predictions': array([11.053898], dtype=float32)}
{'predictions': array([11.096132], dtype=float32)}
{'predictions': array([11.076334], dtype=float32)}
{'predictions': array([11.099128], dtype=float32)}
{'predictions': array([11.096041], dtype=float32)}
{'predictions': array([11.098181], dtype=float32)}
{'predictions': array([11.099011], dtype=float32)}
{'predictions': array([11.097855], dtype=float32)}
{'predictions': array([11.0102

{'predictions': array([11.097091], dtype=float32)}
{'predictions': array([11.09463], dtype=float32)}
{'predictions': array([11.0963745], dtype=float32)}
{'predictions': array([11.076786], dtype=float32)}
{'predictions': array([11.096756], dtype=float32)}
{'predictions': array([11.01373], dtype=float32)}
{'predictions': array([11.098338], dtype=float32)}
{'predictions': array([11.097393], dtype=float32)}
{'predictions': array([11.098237], dtype=float32)}
{'predictions': array([11.097627], dtype=float32)}
{'predictions': array([11.056481], dtype=float32)}
{'predictions': array([11.097729], dtype=float32)}
{'predictions': array([11.097851], dtype=float32)}
{'predictions': array([11.09766], dtype=float32)}
{'predictions': array([11.075158], dtype=float32)}
{'predictions': array([11.098625], dtype=float32)}
{'predictions': array([11.0738], dtype=float32)}
{'predictions': array([11.098567], dtype=float32)}
{'predictions': array([11.098932], dtype=float32)}
{'predictions': array([11.065662], 

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [14]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If i

INFO:tensorflow:loss = 61.35171, step = 6000 (0.422 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 6071...
INFO:tensorflow:Saving checkpoints for 6071 into taxi_trained/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 6071...
INFO:tensorflow:Loss for final step: 61.598927.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-16T15:53:07Z
INFO:tensorflow:Graph was finalized.
INFO

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [1]:
from google.cloud import bigquery
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 4)) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 4)) = {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), {1})) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bigquery.Client().query(query).to_dataframe()

ModuleNotFoundError: No module named 'google.cloud'

In [2]:
print_rmse(model, df)

NameError: name 'print_rmse' is not defined

RMSE on benchmark dataset is <b>9.61</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License